In [1]:
import numpy as np
import pandas as pd
from Branch import Branch
from sklearn.ensemble import RandomForestClassifier
from ConjunctionSet import ConjunctionSet
from sklearn.tree import export_graphviz
import pydot
import matplotlib.pyplot as plt
from DataPreperation import *
from ReadDatasetFunctions import *
from NewModelBuilder import *
import datetime
%matplotlib inline

In [2]:
data,x_columns,y_column = read_iris_data()
train_x,train_y,test_x,test_y=divide_to_train_test(data,x_columns,y_column)

In [3]:
model=RandomForestClassifier(n_estimators=5)
model.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=5, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [4]:
cs=ConjunctionSet(x_columns, data, model, amount_of_branches_threshold=1000, filter_approach='probability')

i=0
i=1
i=2
i=3


In [5]:
df=cs.get_conjunction_set_df().round(decimals=2)

In [6]:
%%prun
node=Node([True]*len(df))
node.split(df)

In [9]:
node.number_of_children()

269

In [ ]:
node.left.left.right.left.check_feature_split_value(df,'0_upper')

In [ ]:
df[node.left.left.right.left.mask]

In [ ]:
df[node.right.mask]

In [ ]:
df[node.right.mask][:3]

In [ ]:
df[node.mask]

In [ ]:
from scipy.stats import entropy
entropy(np.array([1,0,0]))

In [ ]:
test_x[2]

In [10]:
for inst in test_x:
    #print('branch:')
    for i,b in enumerate(cs.conjunctionSet):
        if b.containsInstance(inst):
            probas_sum=np.sum(b.label_probas)
            #print(b.label_probas/probas_sum)
    #print(model.predict_proba(inst)[0])
    print('new model:')
    prediction,depth=node.predict(inst,df)
    print(prediction)
    print(depth)
    print('random forest:')
    print(model.predict_proba(inst)[0])
    print('--------------------')

new model:
[ 1.  0.  0.]
4
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 0.  0.  1.]
7
random forest:
[ 0.  0.  1.]
--------------------
new model:
[ 0.  0.  1.]
7
random forest:
[ 0.  0.  1.]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  0.  1.]
7
random forest:
[ 0.  0.  1.]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 0.  1.  0.]
12
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 0.  0.  1.]
7
random forest:
[ 0.  0.  1.]
-------------------

C:\Users\sagio\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\sagio\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\sagio\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single featu

random forest:
[ 0.   0.2  0.8]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 1.  0.  0.]
6
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.   0.2  0.8]
7
random forest:
[ 0.   0.2  0.8]
--------------------
new model:
[ 1.  0.  0.]
4
random forest:
[ 1.  0.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  1.  0.]
8
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 0.  0.  1.]
7
random forest:
[ 0.  0.  1.]
--------------------
new model:
[ 0.  1.  0.]
12
random forest:
[ 0.  1.  0.]
--------------------
new model:
[ 1. 

C:\Users\sagio\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\sagio\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\sagio\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single featu

In [ ]:
df[node.left.right.left.left.mask]

In [ ]:
node.left.right.left.left.check_feature_split_value(df,'1_lower')

In [ ]:
n=node.left.right.left.left
feature_columns=[col for col in df.columns if 'lower' in col or 'upper' in col]
relevant_cols=[col for col in feature_columns if len(set(df[n.mask][col])) > 1]
relevant_cols

In [ ]:
n=node.left.right.left.left
col='1_lower'
threshold_to_devision_value={}
if 'upper' in col:
    for threshold in set(df[n.mask][col]):
        right_size = len([i for i in df[n.mask][col.replace('upper','lower')] if i > threshold])
        left_size = len(df[n.mask]) - right_size
        threshold_to_devision_value[threshold]=np.abs(left_size-right_size)
else:
    for threshold in set(df[n.mask][col]):
        right_size = len([i for i in df[n.mask][col.replace('lower','upper')] if i <= threshold])
        left_size = len(df[n.mask]) - right_size
        threshold_to_devision_value[threshold] = np.abs(left_size - right_size)
threshold = min(threshold_to_devision_value, key=threshold_to_devision_value.get)
val= threshold_to_devision_value[threshold]
print(threshold,val)

In [ ]:
for inst in test_x:
    for i,b in enumerate(cs.conjunctionSet):
        if b.containsInstance(inst):
            probas_sum=np.sum(b.label_probas)
            print(b.label_probas/probas_sum)
            print(i)
    #print(model.predict_proba(inst)[0])
    print(node.predict(inst,df))
    print('--------------------')

In [ ]:
node.predict(test_x[11],df)

In [ ]:
df

In [ ]:
model.predict_proba(test_x[1])

In [ ]:
model.classes_

In [ ]:
test_y[1]

In [ ]:
model.predict_proba(test_x[1])

In [ ]:
node.predict(test_x[1],df)

In [ ]:
df.sort_values('branch_probability',ascending=False)

In [ ]:
v=np.array([np.array(l) for l in df['probas']])
branches_probas=np.array(df['branch_probability'].values).reshape(len(df),1)
v=v*branches_probas
v=v.mean(axis=0)
v=[i/sum(v) for i in v]
v

In [ ]:
df[node.left.left.left.left.left.left.left.mask].shape

In [ ]:
df[node.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.left.mask]

In [ ]:
n=node.left.left.left.left.left.left.select_random_split_feature(df,[col for col in df.columns if '_' in col])

In [ ]:
n=node.left.left.left.left.left

In [ ]:
for col in df.columns:
    if 'lower' not in col:
        continue
    

In [ ]:
print(node.left.split_feature)
print(node.left.split_value)

In [ ]:
df[node.left.left.mask]

In [ ]:
print(node.left.left.split_feature)
print(node.left.left.split_value)

In [ ]:
df[node.left.left.left.mask]

In [ ]:
print(node.left.left.left.split_feature)
print(node.left.left.left.split_value)

In [ ]:
original_mask=node.left.left.left.mask
left_mask=node.left.left.left.left_mask
for i,j,k,h in zip(original_mask,left_mask,list(np.logical_and(original_mask,left_mask)),original_mask and left_mask):
    print(i)
    print(j)
    print(k)
    print(h)
    print('-------------')

In [ ]:
node.left.left.left.left_mask

In [ ]:
for i in t.value:
    print(np.argmax(i[0]))

In [ ]:
model=new_model_builder.model
t=model.tree_
new_tree_features=new_model_builder.feature_names
new_features=[]
new_thresholds=[]
original_features=[(int(i.split('<')[0]),float(i.split('<')[1])) for i in new_tree_features]
for feature,threshold in zip(t.feature,t.threshold):
    if feature<0:
        new_features.append(feature)
        new_thresholds.append(threshold)
        continue
    new_features.append(original_features[feature][0])
    new_thresholds.append(original_features[feature][1])
new_thresholds

In [ ]:
records=[]
for b in mg.conjunctionSet:
    records.extend(b.get_branch_records())


In [ ]:
data=pd.DataFrame(records).fillna(0)

In [ ]:
est=rf.estimators_[0]
t=est.tree_
export_graphviz(t,out_file='tree.dot',feature_names=iris.feature_names,class_names=iris.target_names,  
                         filled=True, rounded=True)